In [7]:
from dotenv import load_dotenv

load_dotenv("openai.env")

True


tookit是langchain已经封装好的一系列工具，一个工具包是一组工具来组合完成特定的任务
<hr>

# Azure认知服务 https://portal.azure.com/#allservices
- AzureCogsFormRecognizerTool：从文档里提取文本
- AzureCogsSpeech2TextTool：语音到文本
- AzureCogsText2SpeechTool：文本到语音

In [ ]:
! pip install --upgrade azure-ai-formrecognizer
! pip install --upgrade azure-cognitiveservices-speech > /dev/null
! pip install azure-ai-textanalytics

In [4]:
import os

os.environ["AZURE_COGS_KEY"] = "c10"
os.environ["AZURE_COGS_ENDPOINT"] = "https://eastus.api.cognitive.microsoft.com/"
os.environ["AZURE_COGS_REGION"] = "eastus"

In [5]:
#创建toolkit
from langchain.agents.agent_toolkits import AzureCognitiveServicesToolkit

toolkit = AzureCognitiveServicesToolkit()
[tool.name for tool in toolkit.get_tools()]

['azure_cognitive_services_form_recognizer',
 'azure_cognitive_services_speech2text',
 'azure_cognitive_services_text2speech',
 'azure_cognitive_services_text_analyics_health']

In [8]:
#agent使用
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")
agent = initialize_agent(
    tools=toolkit.get_tools(),
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

/Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [9]:
agent.invoke("Tell me a joke and read it out for me.")



> Entering new AgentExecutor chain...
Thought: I will provide a joke and then use the text-to-speech service to read it out loud.

Action:
```json
{
  "action": "azure_cognitive_services_text2speech",
  "action_input": {
    "query": "Why don't scientists trust atoms? Because they make up everything!"
  }
}
```


RuntimeError: Error while running AzureCogsText2SpeechTool: Speech synthesis error: WebSocket upgrade failed: Authentication error (401). Please check subscription information and region name. USP state: Sending. Received audio size: 0 bytes.

# python代码机器人


In [10]:
! pip install langchain_experimental

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 1.9 MB/s eta 0:00:00a 0:00:01


In [11]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools import PythonREPLTool
from langchain_experimental.utilities import PythonREPL
from langchain.llms.openai import OpenAI
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

agent_executor = create_python_agent(
    llm=ChatOpenAI(temperature=0, model="gpt-4-1106-preview"),
    tool=PythonREPLTool(),
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    agent_executor_kwargs={"handle_parsing_errors": True},
)

agent_executor.invoke("What is the 10th fibonacci number?")



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.



Invoking: `Python_REPL` with `def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(10))`


55
The 10th Fibonacci number is 55.

> Finished chain.


{'input': 'What is the 10th fibonacci number?',
 'output': 'The 10th Fibonacci number is 55.'}

In [12]:
agent_executor.invoke(
    """Understand, write a single neuron neural network in PyTorch.
Take synthetic data for y=2x. Train for 100 epochs and print every 10 epochs.
Return prediction for x = 5"""
)



> Entering new AgentExecutor chain...

Invoking: `Python_REPL` with `import torch
import torch.nn as nn
import torch.optim as optim

# Define a single neuron neural network
class SingleNeuron(nn.Module):
    def __init__(self):
        super(SingleNeuron, self).__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

# Create synthetic data for y = 2x
X = torch.arange(-10, 11, dtype=torch.float32).view(-1, 1)
Y = 2 * X

# Initialize the model
model = SingleNeuron()

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model
for epoch in range(100):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, Y)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')

# Predict for x = 5
x_test = torch.tensor([[5.0]])
prediction = model(

{'input': 'Understand, write a single neuron neural network in PyTorch.\nTake synthetic data for y=2x. Train for 100 epochs and print every 10 epochs.\nReturn prediction for x = 5',
 'output': 'After training the single neuron neural network for 100 epochs on the synthetic data for y=2x, the prediction for x=5 is approximately 9.8781. Here are the loss values printed every 10 epochs:\n\n- Epoch [10/100], Loss: 0.5877\n- Epoch [20/100], Loss: 0.3924\n- Epoch [30/100], Loss: 0.2619\n- Epoch [40/100], Loss: 0.1749\n- Epoch [50/100], Loss: 0.1167\n- Epoch [60/100], Loss: 0.0779\n- Epoch [70/100], Loss: 0.0520\n- Epoch [80/100], Loss: 0.0347\n- Epoch [90/100], Loss: 0.0232\n- Epoch [100/100], Loss: 0.0155'}

# SQL Database
使用SQLDatabaseChain构建的agent，用来根据数据库回答问题

In [15]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

agent_executor = create_sql_agent(
    llm=ChatOpenAI(temperature=0, model="gpt-4-1106-preview"),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS
)

In [16]:
agent_executor.invoke("描述一下playlisttrack这张表")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'PlaylistTrack'}`



CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/The `PlaylistTrack` table is a junction table that associates tracks with playlists. It has two columns:

1. `PlaylistId`: An integer that is a foreign key referencing the `Playlist` table. It is part of the composite primary key for the `PlaylistTrack` table.
2. `TrackId`: An integer that is a foreign key referencing the `Track` table. It is also part of the composite primary key for the `Playli

{'input': '描述一下playlisttrack这张表',
 'output': 'The `PlaylistTrack` table is a junction table that associates tracks with playlists. It has two columns:\n\n1. `PlaylistId`: An integer that is a foreign key referencing the `Playlist` table. It is part of the composite primary key for the `PlaylistTrack` table.\n2. `TrackId`: An integer that is a foreign key referencing the `Track` table. It is also part of the composite primary key for the `PlaylistTrack` table.\n\nThe primary key of the table is a composite key made up of both `PlaylistId` and `TrackId`, ensuring that the combination of a playlist and a track is unique within the table.\n\nHere are three sample rows from the `PlaylistTrack` table:\n\n| PlaylistId | TrackId |\n|------------|---------|\n| 1          | 3402    |\n| 1          | 3389    |\n| 1          | 3390    |\n\nThese rows indicate that the tracks with `TrackId` 3402, 3389, and 3390 are all part of the playlist with `PlaylistId` 1.'}

In [17]:
# 用自然语言跑sql查询
agent_executor.invoke(
    "Show the total number of tracks in each playlist. The Playlist name should be included in the result."
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Playlist, PlaylistTrack'}`



CREATE TABLE "Playlist" (
	"PlaylistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("PlaylistId")
)

/*
3 rows from Playlist table:
PlaylistId	Name
1	Music
2	Movies
3	TV Shows
*/


CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT p.Name AS PlaylistName, COUNT(pt.TrackId) AS TotalTracks FROM Playlist p JOIN PlaylistTrack pt ON p.PlaylistId = pt.PlaylistId GROUP BY p

{'input': 'Show the total number of tracks in each playlist. The Playlist name should be included in the result.',
 'output': 'Here are the total number of tracks in each playlist, along with the playlist names:\n\n- Music: 3290 tracks\n- TV Shows: 213 tracks\n- 90’s Music: 1477 tracks\n- Music Videos: 1 track\n- Brazilian Music: 39 tracks\n- Classical: 75 tracks\n- Classical 101 - Deep Cuts: 25 tracks\n- Classical 101 - Next Steps: 25 tracks\n- Classical 101 - The Basics: 25 tracks\n- Grunge: 15 tracks\n- Heavy Metal Classic: 26 tracks\n- On-The-Go 1: 1 track\n\n(Note: Some playlist names may be duplicated in the result due to the way the data is stored in the database.)'}

In [19]:
# 会自动会找表查询结果
agent_executor.invoke("Who are the top 3 best selling artists?")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Artist, InvoiceLine, Track, Album'}`



CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "InvoiceLine" (
	"InvoiceLineId" INTEGER NOT NULL, 
	"InvoiceId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	"Quantity" INTEG

{'input': 'Who are the top 3 best selling artists?',
 'output': 'The top 3 best-selling artists are:\n\n1. Iron Maiden with 140 sales\n2. U2 with 107 sales\n3. Metallica with 91 sales'}